# Интеграция алгоритмов поиска в систему DEGANN

## 📌 Контекст и проблема
**Последние изменения в системе:**
- Полностью переработана система конфигураций нейросетевых топологий
- Добавлен механизм метаданных для настройки параметров
- Введена новая структура хранения параметров обучения

**Возникшие сложности:**
- Существующие алгоритмы поиска (grid search и др.) перестали работать
- Невозможно быстро добавлять новые типы архитектур
- Код алгоритмов "завязан" на конкретные реализации топологий

**Почему это критично:**
- Система теряет гибкость — ключевое требование для DEGANN
- Замедляется добавление экспериментальных архитектур (GAN, Transformer)

In [1]:
import sys
from pathlib import Path
import os

# 1. Установка зависимостей из вашего requirements.txt
%pip install -q optuna cmaes tensorflow matplotlib numpy scikit-learn

# 2. Клонирование ВАШЕГО репозитория (с подмодулем)
if not Path("degann-hpo-prototype").exists():
    %git clone --recursive https://github.com/heroi17/degann-hpo-prototype.git
    os.chdir("degann-hpo-prototype")  # Переход в папку проекта

    # 3. Фиксация конкретного коммита DEGANN (важно!)
    %cd deps/degann && git checkout 9727df0

# 4. Проверка путей
degann_path = Path("deps/degann").absolute()
sys.path.append(str(degann_path))

# 5. Проверка импорта
try:
    from degann.networks import IModel
    print("✅ DEGANN успешно импортирован!")
    print(f"Путь: {degann_path}")
    print(f"Версия: 9727df0")
except ImportError as e:
    print(f"❌ Ошибка: {e}")
    print("Советы:")
    print("1. Убедитесь, что подмодуль загружен: `git submodule update --init`")
    print("2. Проверьте коммит в deps/degann: `git log -1`")


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
✅ DEGANN успешно импортирован!
Путь: c:\Users\Mikhail\Desktop\degann\degann-hpo-prototype\deps\degann
Версия: 9727df0



---

## 🧪 Тестовый пример с Optuna
**Цель:** Проверить базовую интеграцию Optuna до работы с DEGANN

In [2]:
import optuna
import math

def objective(trial):
    """Тестовая функция для проверки работы Optuna."""
    x = trial.suggest_float("x", -10, 10)
    y = trial.suggest_float("y", -10, 10)
    return math.sin(x) + math.cos(y)

# Запуск оптимизации с разными алгоритмами
samplers = {
    "CMA-ES": optuna.samplers.CmaEsSampler(),
    "TPE": optuna.samplers.TPESampler(),
    "Random": optuna.samplers.RandomSampler()
}

for name, sampler in samplers.items():
    study = optuna.create_study(direction="maximize", sampler=sampler)
    study.optimize(objective, n_trials=50)
    print(f"{name}: Best value = {study.best_value:.3f}")

[I 2025-06-08 14:36:46,111] A new study created in memory with name: no-name-de7a9d76-e01c-425a-9753-6398aab79e41
[I 2025-06-08 14:36:46,113] Trial 0 finished with value: 1.877129881243738 and parameters: {'x': -4.427040436225489, 'y': 5.874304282764523}. Best is trial 0 with value: 1.877129881243738.
[I 2025-06-08 14:36:48,426] Trial 1 finished with value: 0.5784326027733383 and parameters: {'x': 0.41736731907532487, 'y': -4.886342384687463}. Best is trial 0 with value: 1.877129881243738.
[I 2025-06-08 14:36:48,427] Trial 2 finished with value: 0.5589343362938161 and parameters: {'x': -0.3941724753022964, 'y': -0.3393278948257148}. Best is trial 0 with value: 1.877129881243738.
[I 2025-06-08 14:36:48,428] Trial 3 finished with value: 0.4769807921415369 and parameters: {'x': -0.4223322454968468, 'y': 6.763456450448043}. Best is trial 0 with value: 1.877129881243738.
[I 2025-06-08 14:36:48,430] Trial 4 finished with value: -1.8946033319212168 and parameters: {'x': -1.9180327672397297, '

CMA-ES: Best value = 1.989


[I 2025-06-08 14:36:48,672] Trial 41 finished with value: 1.580164191125117 and parameters: {'x': 8.516292842225154, 'y': 6.940573122220375}. Best is trial 6 with value: 1.8827376235329085.
[I 2025-06-08 14:36:48,677] Trial 42 finished with value: 0.15879741887936194 and parameters: {'x': 9.97933912663327, 'y': 7.0988731052117355}. Best is trial 6 with value: 1.8827376235329085.
[I 2025-06-08 14:36:48,682] Trial 43 finished with value: 0.545951169918633 and parameters: {'x': 8.167194298714524, 'y': 8.271395187268478}. Best is trial 6 with value: 1.8827376235329085.
[I 2025-06-08 14:36:48,687] Trial 44 finished with value: 0.5432206529443306 and parameters: {'x': 6.790801385212613, 'y': 4.76954544174116}. Best is trial 6 with value: 1.8827376235329085.
[I 2025-06-08 14:36:48,692] Trial 45 finished with value: -1.8061688506501645 and parameters: {'x': 4.15495975037257, 'y': 3.434006002040363}. Best is trial 6 with value: 1.8827376235329085.
[I 2025-06-08 14:36:48,697] Trial 46 finished w

TPE: Best value = 1.883
Random: Best value = 1.868



---

## 🚀 Переход к реальной задаче
**Цель:** Применить Optuna для поиска оптимальной GAN-архитектуры в DEGANN.

**План эксперимента:**
1. Подготовить тестовые данные (функция `sin(10x)`)
2. Определить изменяемые параметры:
   - Размеры слоев генератора/дискриминатора
   - Функции активации
   - Learning rate
3. Реализовать objective-функцию для Optuna
4. Запустить поиск с сохранением лучшей модели

**Ключевое отличие от теста:**
- Оптимизируем не детерминированную функцию, а стохастический процесс обучения
- Каждое "вычисление" — это полноценное обучение нейросети


In [3]:
# Основные импорты
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Компоненты DEGANN
from degann.networks import IModel
from degann.networks.topology.densenet.topology_config import DenseNetParams
from degann.networks.topology.densenet.compile_config import DenseNetCompileParams
from degann.networks.topology.gan.topology_config import GANTopologyParams
from degann.networks.topology.gan.compile_config import GANCompileParams

# Генерация данных
def generate_data(n_samples=2048):
    """Создает данные для обучения GAN (аппроксимация sin(10x))."""
    X = np.linspace(0, 1, int(n_samples / 0.8)).reshape(-1, 1)
    y = np.sin(10 * X)
    return train_test_split(X, y, test_size=0.2)

X_train, X_test, y_train, y_test = generate_data()

In [4]:
# Временное хранилище лучшей модели (заменить на класс-менеджер в продакшене)
best_gan = None  # TODO: Заменить на механизм persistent-хранилища
best_score = float('inf')

def create_gan_config(trial):
    """
    Генерация конфига GAN на основе параметров из Optuna
    
    Args:
        trial: Объект Optuna Trial для запроса параметров
        
    Returns:
        Словарь с параметрами:
        - gen_layers: количество слоев генератора (1-5)
        - gen_neurons: нейроны в слоях [16, 32, 64]
        - ... и т.д.
    """
    # Параметры генератора
    gen_layers = trial.suggest_int('gen_layers', 1, 5)
    gen_neurons = trial.suggest_categorical('gen_neurons', [16, 32, 64])
    
    # Параметры дискриминатора
    disc_layers = trial.suggest_int('disc_layers', 1, 5)
    disc_neurons = trial.suggest_categorical('disc_neurons', [16, 32, 64])
    
    # Обучение
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True)
    
    return {
        'gen_layers': gen_layers,
        'gen_neurons': gen_neurons,
        'disc_layers': disc_layers,
        'disc_neurons': disc_neurons,
        'learning_rate': learning_rate
    }

In [5]:
def gan_objective(trial):
    """Функция для оптимизации средствами Optuna."""
    global best_gan, best_score
    
    # 1. Получаем параметры
    params = create_gan_config(trial)
    
    # 2. Собираем GAN
    gan = build_gan(params)
    
    # 3. Обучение (упрощенное)
    history = gan.train(
        X_train, y_train,
        epochs=100,
        mini_batch_size=64,
        verbose=0
    )
    
    # 4. Оценка
    val_loss = gan.evaluate(X_test, y_test, verbose=0, return_dict=True)["root_mean_squared_error"]
    
    # 5. Сохранение лучшей модели
    if val_loss < best_score:
        best_score = val_loss
        best_gan = gan
        
    return val_loss

def build_gan(params: dict) -> IModel:
    """Создает и компилирует GAN модель на основе параметров.
    
    Args:
        params: Словарь с параметрами из Optuna:
            - gen_layers: int - число слоев генератора
            - gen_neurons: int - нейроны в слоях генератора
            - disc_layers: int - число слоев дискриминатора 
            - disc_neurons: int - нейроны в слоях дискриминатора
            - learning_rate: float - скорость обучения
            
    Returns:
        Скомпилированная GAN модель (IModel)
    """
    # 1. Конфиг генератора
    gen_config = DenseNetParams(
        input_size=1,
        block_size=[params['gen_neurons']] * params['gen_layers'],
        output_size=1,
        activation_func="leaky_relu"  # Можно сделать tunable
    )
    
    # 2. Конфиг дискриминатора
    disc_config = DenseNetParams(
        input_size=2,
        block_size=[params['disc_neurons']] * params['disc_layers'],
        output_size=1,
        activation_func="leaky_relu"
    )
    
    # 3. Сборка GAN
    gan_params = GANTopologyParams(
        generator_params=gen_config,
        discriminator_params=disc_config
    )
    
    # 4. Компиляция
    compile_config = GANCompileParams(
        generator_params=DenseNetCompileParams(
            rate=params['learning_rate'],
            optimizer="Adam",
            loss_func="BinaryCrossentropy"
        ),
        discriminator_params=DenseNetCompileParams(
            rate=params['learning_rate'], 
            optimizer="Adam",
            loss_func="BinaryCrossentropy"
        )
    )
    
    # 5. Создание модели
    gan = IModel(gan_params)
    compile_config.add_eval_metric("root_mean_squared_error")
    gan.compile(compile_config)
    
    return gan

In [6]:
# Закоментируйте это если хотите более подробный вывод истории подбора параметров
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Создание вспомогательных функций для вывода
def print_summary(study):
    """Красивый вывод результатов."""
    print("\n" + "="*50)
    print("🎯 Best trial:")
    for k, v in study.best_trial.params.items():
        print(f"- {k:15}: {v}")
    print(f"\n🏆 Best value: {study.best_value:.4f}")
    print("="*50)

def smart_callback(study, trial):
    """Умный вывод: только каждые 5 trials и улучшения."""
    if trial.number % 5 == 0 or study.best_trial.number == trial.number:
        msg = f"Trial {trial.number}: val_loss={trial.value:.4f}"
        if study.best_trial.number == trial.number:
            msg += " 🏆 NEW BEST"
        print(msg)


In [7]:

# Настройка исследования
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler()
)

# Лимит: 50 trials или 1 час
study.optimize(
    gan_objective, 
    n_trials=20, 
    timeout=3600,
    callbacks=[smart_callback],
    show_progress_bar=True
)

[I 2025-06-08 14:36:49,761] A new study created in memory with name: no-name-6df1f5e3-e3c2-40a0-9f93-ff5d3b1d1344


  0%|          | 0/20 [00:00<?, ?it/s]


[I 2025-06-08 14:36:56,883] Trial 0 finished with value: 0.8371961712837219 and parameters: {'gen_layers': 2, 'gen_neurons': 32, 'disc_layers': 2, 'disc_neurons': 16, 'learning_rate': 0.00037458176163169556}. Best is trial 0 with value: 0.8371961712837219.
Trial 0: val_loss=0.8372 🏆 NEW BEST
[I 2025-06-08 14:37:03,715] Trial 1 finished with value: 2.6797680854797363 and parameters: {'gen_layers': 3, 'gen_neurons': 16, 'disc_layers': 5, 'disc_neurons': 16, 'learning_rate': 4.180900965971247e-05}. Best is trial 0 with value: 0.8371961712837219.
[I 2025-06-08 14:37:09,836] Trial 2 finished with value: 2.1859304904937744 and parameters: {'gen_layers': 3, 'gen_neurons': 32, 'disc_layers': 3, 'disc_neurons': 64, 'learning_rate': 0.0006049207674566109}. Best is trial 0 with value: 0.8371961712837219.
[I 2025-06-08 14:37:15,766] Trial 3 finished with value: 2.6206367015838623 and parameters: {'gen_layers': 3, 'gen_neurons': 16, 'disc_layers': 3, 'disc_neurons': 32, 'learning_rate': 0.00057141

In [8]:
print_summary(study)

if best_gan:
    print("Модель сохранена в переменной `best_gan`")


🎯 Best trial:
- gen_layers     : 1
- gen_neurons    : 32
- disc_layers    : 1
- disc_neurons   : 16
- learning_rate  : 7.081103266813977e-05

🏆 Best value: 0.7012
Модель сохранена в переменной `best_gan`


## 🔎 Выводы из эксперимента с GAN

### Что получилось:
✅ **Интеграция Optuna работает**:  
   - Алгоритм успешно подбирает архитектуру (слои, нейроны, LR)  
   - Лучшая модель сохраняется автоматически  

✅ **Доказана концепция**:  
   - DEGANN + Optuna решают задачу оптимизации **параметров сети**  
   - **Шаблон кода** (objective + обучение) универсален, но требует:  
     - Реализации `build_model()` под конкретную архитектуру  
     - Ручного описания параметров (`suggest_int`, `suggest_float`) 

### Проблемы текущей реализации:
⚠️ **Жёсткая привязка к GAN**:  
   - Параметры прописаны вручную (`gen_layers`, `disc_neurons`)  
   - Нет унификации для других архитектур  

⚠️ **Ограниченный поиск**:  
   - Нельзя динамически менять:  
     - Число слоёв  
     - Типы слоёв (например, LSTM + Dense)  

### Требуется улучшение:
1. **Абстрактный слой** для описания параметров (→ `ConfigSampler`)  
2. **Поддержка сложных правил** (например, "от 2 до 5 слоёв с шагом 16 нейронов")  
3. **Отделение логики поиска** от обучения модели  


---

## 🛠️ Реализация ConfigSampler

### Концепция
ConfigSampler - прослойка между:
1. Конфигами DEGANN (с метаданными tunable-параметров)
2. Алгоритмами поиска (Optuna и др.)

Решает:
- Автоматическое обнаружение параметров для оптимизации
- Преобразование "плоских" параметров поиска в сложные конфиги

### Основная реализация

In [9]:
from copy import deepcopy
from dataclasses import fields, is_dataclass
from typing import Any, Dict, List, Union, TypeVar, Generic
from degann.networks.topology.tuning_utils import TuningMetadata, FieldMetadata

T = TypeVar('T')

class ConfigSampler(Generic[T]):
    """
    Bridges between configuration space and search algorithms.
    
    Responsibilities:
    1. Discovers tunable parameters via field metadata
    2. Transforms search algorithm's output into valid configurations
    
    Usage:
    >>> sampler = ConfigSampler(config)
    >>> concrete_config = sampler.sample_config({"param1": value1})
    """
    
    def __init__(self, generic_config: T):
        # Глубокое копирование исходного конфига
        self.generic_config = deepcopy(generic_config)
        self.variable_fields = self._find_variable_fields(self.generic_config)

    def _find_variable_fields(self, config: Any, prefix: str = "") -> Dict[str, FieldMetadata]:
        """Рекурсивно собирает все варьируемые поля и их метаданные."""
        variables = {}
        
        if not is_dataclass(config) or not hasattr(config, "tuning_metadata"):
            return variables
        
        tuning_metadata: TuningMetadata = config.tuning_metadata
        
        for field in fields(config):
            field_name = f"{prefix}{field.name}"
            field_value = getattr(config, field.name)
            
            # Поле помечено как tunable?
            if field.metadata.get("tunable", False):
                meta = tuning_metadata.get(field.name, None)
                if meta:
                    variables[field_name] = meta
            
            # Рекурсия для вложенных конфигов
            if is_dataclass(field_value):
                nested_vars = self._find_variable_fields(field_value, f"{field_name}.")
                variables.update(nested_vars)
        
        return variables

    def sample_config(self, values: Dict[str, Any]) -> T:
        """Создаёт конкретный конфиг на основе выбранных значений."""
        config_copy = deepcopy(self.generic_config)
        return self._apply_values(config_copy, values)

    def _apply_values(self, config: Any, values: Dict[str, Any], prefix: str = "") -> Any:
        """Рекурсивно применяет значения к варьируемым полям."""
        if not is_dataclass(config):
            return config
            
        for field in fields(config):
            field_name = f"{prefix}{field.name}"
            field_value = getattr(config, field.name)
            
            if field_name in values:
                setattr(config, field.name, values[field_name])
            elif is_dataclass(field_value):
                nested_config = self._apply_values(field_value, values, f"{field_name}.")
                setattr(config, field.name, nested_config)
        
        return config

## Тест базовой функциональности

In [10]:
# Минимальный тест-конфиг для проверки ядра функциональности
from dataclasses import dataclass, field, InitVar
from typing import Union, Any, Optional


@dataclass 
class TestConfig:
    layers: int = field(default="1", metadata={"tunable": True})
    activation: str = field(default="relu", metadata={"tunable": True})
    metadata: InitVar[dict | None] = None
    tuning_metadata: Optional[TuningMetadata] = field(default=None, init=False)

    def __init__(self):
        metadata={
            "layers": FieldMetadata(
                length_boundary=(3, 3)
            ),
            "activation": FieldMetadata(
                choices=["tanh"]
           )
        }
        self.tuning_metadata = TuningMetadata(type(self))
        self.tuning_metadata.set_metadata(metadata)

# Проверка основных сценариев
def test_sampler_basic():
    # 1. Инициализация
    sampler = ConfigSampler(TestConfig())
    
    # 2. Корректное применение значений
    assert sampler.variable_fields.__contains__("layers")
    assert sampler.variable_fields.__contains__("activation")

    config = sampler.sample_config({"layers": 3, "activation": "tanh"})

    assert config.layers == 3
    assert config.activation == "tanh"
    
test_sampler_basic()

## 🧪 Практическая проверка: интеграция с GAN

### 1. Подготовка тестовых данных и конфигураций

In [11]:
# Глобальные переменные для отслеживания лучшей модели
best_score = float("inf")
best_model = None

# Фиксированные параметры для теста
GEN_LAYERS = 4
GEN_NEURONS = 16  # не более 64
DISC_LAYERS = 4
DISC_NEURONS = 32

def create_gan_compile_config(learning_rate=1e-4):
    """Создает фиксированную конфигурацию компиляции для GAN"""
    return GANCompileParams(
        generator_params=DenseNetCompileParams(
            rate=learning_rate,
            optimizer="Adam",
            loss_func="BinaryCrossentropy",
            metric_funcs=["mean_absolute_error"],
        ),
        discriminator_params=DenseNetCompileParams(
            rate=learning_rate,
            optimizer="Adam",
            loss_func="BinaryCrossentropy",
            metric_funcs=["binary_accuracy"],
        )
    )

### 2. Определение конфигурации GAN с tunable параметрами

In [12]:
def create_tunable_gan_config():
    """Создает базовую конфигурацию GAN с метаданными для поиска"""
    # Конфиг генератора
    gen_config = DenseNetParams(
        input_size=1,
        block_size=[GEN_NEURONS] * GEN_LAYERS,
        output_size=1,
        activation_func="leaky_relu",
        metadata={
            "block_size": FieldMetadata(
                choices=[
                    [16, 16],       # 2 слоя
                    [32, 64, 32],   # 3 слоя
                    [16, 32, 32, 16] # 4 слоя
                ]
            ),
            "activation_func": FieldMetadata(
                choices=["relu", "leaky_relu", "sigmoid", "tanh"]
            )
        }
    )

    # Конфиг дискриминатора
    disc_config = DenseNetParams(
        input_size=2,
        block_size=[DISC_NEURONS] * DISC_LAYERS,
        output_size=1,
        activation_func="leaky_relu",
        metadata={
            "block_size": FieldMetadata(
                choices=[
                    [16, 16],
                    [32, 64, 32],
                    [16, 32, 32, 16]
                ]
            ),
            "activation_func": FieldMetadata(
                choices=["relu", "leaky_relu", "sigmoid"]
            )
        }
    )

    return GANTopologyParams(
        generator_params=gen_config,
        discriminator_params=disc_config
    )

### 3. Objective-функция для Optuna

In [13]:
import optuna
from degann.networks.topology.base_topology_configs import BaseTopologyParams

def gan_objective(trial: optuna.Trial, generic_config: BaseTopologyParams) -> float:
    """Функция для оптимизации архитектуры GAN"""
    global best_score, best_model
    
    # 1. Инициализация семплера
    sampler = ConfigSampler(generic_config)
    
    # 2. Подбор параметров через Optuna
    values = {}
    for field_name, field_meta in sampler.variable_fields.items():
        if field_meta.value_range:
            min_val, max_val, step = field_meta.value_range
            if isinstance(min_val, int):
                values[field_name] = trial.suggest_int(field_name, min_val, max_val, step)
            else:
                values[field_name] = trial.suggest_float(field_name, min_val, max_val, step)
        elif field_meta.choices:
            values[field_name] = trial.suggest_categorical(field_name, field_meta.choices)
    
    # 3. Создание конкретной конфигурации
    concrete_config = sampler.sample_config(values)
    concrete_compile_config = create_gan_compile_config()
    
    # 4. Создание и обучение модели
    model = IModel(concrete_config)
    concrete_compile_config.add_eval_metric("root_mean_squared_error")
    model.compile(concrete_compile_config)
    
    history = model.train(
        X_train, y_train,
        epochs=100,
        mini_batch_size=64,
        verbose=0
    )
    
    # 5. Оценка и сохранение лучшей модели
    val_loss = model.evaluate(X_test, y_test, verbose=0, return_dict=True)["root_mean_squared_error"]
    
    if val_loss < best_score:
        best_score = val_loss
        best_model = model
    
    return val_loss


### 4. Запуск оптимизации

In [14]:
# Инициализация базового конфига
base_config = create_tunable_gan_config()

# Настройка и запуск исследования Optuna
study = optuna.create_study(direction="minimize")
study.optimize(
    lambda trial: gan_objective(trial, base_config),
    n_trials=3,
    show_progress_bar=True,
    callbacks=[smart_callback]
)

print(f"Лучшая достигнутая ошибка: {best_score:.4f}")

[I 2025-06-08 14:39:10,476] A new study created in memory with name: no-name-cb10eb4d-7f68-41d1-b3f8-871914bebb0d


  0%|          | 0/3 [00:00<?, ?it/s]

c:\Users\Mikhail\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [16, 16] which is of type list.
  warnings.warn(message)
c:\Users\Mikhail\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [32, 64, 32] which is of type list.
  warnings.warn(message)
c:\Users\Mikhail\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [16, 32, 32, 16] which is of type list.
  warnings.warn(message)


[I 2025-06-08 14:39:17,527] Trial 0 finished with value: 0.8631380200386047 and parameters: {'generator_params.block_size': [16, 32, 32, 16], 'generator_params.activation_func': 'leaky_relu', 'discriminator_params.block_size': [16, 32, 32, 16], 'discriminator_params.activation_func': 'leaky_relu'}. Best is trial 0 with value: 0.8631380200386047.
Trial 0: val_loss=0.8631 🏆 NEW BEST
[I 2025-06-08 14:39:24,122] Trial 1 finished with value: 13.721842765808105 and parameters: {'generator_params.block_size': [16, 32, 32, 16], 'generator_params.activation_func': 'leaky_relu', 'discriminator_params.block_size': [16, 16], 'discriminator_params.activation_func': 'relu'}. Best is trial 0 with value: 0.8631380200386047.
[I 2025-06-08 14:39:31,719] Trial 2 finished with value: 44.63447952270508 and parameters: {'generator_params.block_size': [16, 32, 32, 16], 'generator_params.activation_func': 'leaky_relu', 'discriminator_params.block_size': [16, 32, 32, 16], 'discriminator_params.activation_func'


---

## 💡 Future Improvements

### 🔧 Проблема текущей реализации
```python
# Требуется явно перечислять все варианты
metadata = {
    "block_size": FieldMetadata(choices=[
        [16, 16], 
        [32, 64, 32],  # Невозможно выразить правила генерации
        [16, 32, 32, 16]
    ])
}
```


### 🛠️ Предлагаемое решение: Иерархические диапазоны

#### Архитектура:
```mermaid
graph LR
    A[ConfigSampler] --> B[SmartRange]
    B --> C[decompose]
    B --> D[compose]
    C --> E[BasicRange]
    C --> F[Другие SmartRange]
    D --> G[Конкретное значение]
```


#### Пример с block_size:
1. **Декомпозиция**:
   - `SmartRange` раскладывается на 6 элементарных параметров.
   - Например, `block_size` → `num_layers` (1-5) + `neuron_base_1` ... `neuron_base_5` (1-6).
2. **Генерация значений**:
   - Optuna подбирает `num_layers=3`, `neuron_base_1=2`, `neuron_base_2=4`, `neuron_base_3=1`.
3. **Композиция**:
   - Берем первые 3 значения: `[2^2, 2^4, 2^1]` → `[4, 16, 2]`.
   - Возвращаем конкретный `block_size=[4, 16, 2]` в конфиг.

```mermaid
graph LR
    B[Исходный конфиг] --> C[Поле: block_size = SmartRange]
    C --> D[Декомпозиция]
    D --> D1[Длина списка: 1-5]
    D --> D2[Значение 1: 2^1-2^6]
    D --> D3[Значение 2: 2^1-2^6]
    D --> D4[Значение 3: 2^1-2^6]
    D --> D5[Значение 4: 2^1-2^6]
    D --> D6[Значение 5: 2^1-2^6]
    
    D1 --> E[Алгоритм поиска: x=3]
    D2 --> F[Алгоритм поиска: y1=2]
    D3 --> G[Алгоритм поиска: y2=4]
    D4 --> H[Алгоритм поиска: y3=1]
    D5 -.-> I1[Не используется]
    D6 -.-> I2[Не используется]
    
    E & J & K & L & I1 & I2 --> I[Композиция: n=3]
    
    F --> J[Преобразование: 2^2=4]
    G --> K[Преобразование: 2^4=16]
    H --> L[Преобразование: 2^1=2]
    
    I --> M[Результат: 4, 16, 2]
    M --> N[попадает в Конкретный конфиг]

```

#### Реализация:
```python
class SmartRange:
    """Базовый класс для сложных диапазонов"""
    @abstractmethod
    def decompose(self) -> Dict[str, Union[BasicRange, SmartRange]]:
        """Разложение на компоненты"""
        
    @abstractmethod 
    def compose(self, values: Dict[str, Any]) -> Any:
        """Сборка значения из компонентов"""

class LayerSizeRange(SmartRange):
    """Пример: Динамическое создание архитектуры сети"""
    def __init__(self, len_range: tuple, neuron_step: int):
        self.len_range = BasicRange(*len_range)
        self.neuron_step = neuron_step

    def decompose(self):
        return {
            "num_layers": self.len_range,
            "base_neurons": BasicRange(16, 64, self.neuron_step)
        }

    def compose(self, values):
        return [values["base_neurons"]] * values["num_layers"]

```

### ⚠️ Защита от рекурсии

```python
class RecursionGuard:
    def __init__(self):
        self._seen = set()

    def check(self, obj):
        obj_id = id(obj)
        if obj_id in self._seen:
            raise RecursionError("Обнаружена циклическая зависимость")
        self._seen.add(obj_id)

# Использование в ConfigSampler
guard = RecursionGuard()
for field in fields(config):
    guard.check(field.metadata["range"])  # Проверка перед обработкой
```

### 🔄 Пример работы

## Конфигурация с умным диапазоном
```python
gen_config = DenseNetParams(
    block_size=LayerSizeRange((2,5), 16),  # От 2 до 5 слоев по 16 нейронов
    ...
)
```

## Внутри ConfigSampler:
1. Вызов layer_range.decompose() → {"num_layers": (2,5), "base_neurons": (16,64,16)}
2. Подбор значений алгоритмом поиска
3. layer_range.compose({"num_layers":3, "base_neurons":32}) → [32, 32, 32]


---

### 🔧 **Функция `generate_sample` - Сердце трансформации параметров**

**Контекст:**  
ConfigSampler обнаруживает tunable-параметры, но нужен механизм для:  
1. Преобразования "плоских" значений от алгоритма поиска → в сложные конфиги  
2. Поддержки разных типов параметров (числа, категории, диапазоны)  

**Решение:**  
```python
def generate_sample(trial: optuna.Trial, sampler: ConfigSampler) -> Any:
    """Связующее звено между алгоритмом поиска и DEGANN-конфигами"""
    # 1. Собираем значения для всех tunable-полей через trial.suggest_*
    # 2. Применяем их к базовому конфигу через sampler.sample_config()
```

**Реализация:**

In [15]:
import optuna
from degann.networks.topology.base_topology_configs import BaseTopologyParams

def generate_sample(trial: optuna.Trial, sampler: ConfigSampler) -> Any:
    """
    Генерирует конкретный конфиг на основе параметров из алгоритма поиска.
    
    Args:
        trial: Объект Optuna Trial для подбора параметров.
        sampler: Экземпляр ConfigSampler с информацией о tunable-полях.
        
    Returns:
        Конкретный конфиг с подобранными параметрами.
        
    Процесс:
        1. Для каждого tunable-поля в конфиге:
           - Если задан диапазон (value_range), подбирает целое/вещественное значение.
           - Если заданы варианты (choices), выбирает категориальное значение.
        2. Создает конфиг с подобранными значениями.
    """
    values = {}
    for field_name, field_meta in sampler.variable_fields.items():
        if field_meta.value_range:
            min_val, max_val, step = field_meta.value_range
            if isinstance(min_val, int):
                values[field_name] = trial.suggest_int(field_name, min_val, max_val, step)
            else:
                values[field_name] = trial.suggest_float(field_name, min_val, max_val, step)
        elif field_meta.choices:
            values[field_name] = trial.suggest_categorical(field_name, field_meta.choices)
    
    return sampler.sample_config(values)


**Ключевые особенности:**  
✅ **Универсальность**  
- Работает с любым алгоритмом поиска (Optuna, CMA-ES, etc)  
- Поддерживает все типы параметров через FieldMetadata  

✅ **Инкапсуляция**  
- Скрывает сложность преобразования параметров  
- Не требует модификации при добавлении новых полей  



---

### 🧩 **Класс `HyperparameterSearch` - Фабрика оптимизации**

**Проблема:**  
Ручное управление:  
- Конфигами топологии/компиляции  
- Данными обучения  
- Треккингом лучшей модели  

**Архитектура решения:**  
```mermaid
graph LR
    A[Optuna] --> B[HyperparameterSearch]
    B --> C[ConfigSampler]
    B --> D[IModel]
    B --> E[Треккинг лучшей модели]
```

**Ключевые методы:**  
| Метод                | Роль                                                                 |
|----------------------|----------------------------------------------------------------------|
| `generate_configs()` | Создает связанные конфиги топологии и компиляции                     |
| `train_model()`      | Обучает модель и вычисляет метрику                                   |
| `get_objective()`    | Генерирует функцию для Optuna (интеграционная точка)                 |


**Реализация:**



In [16]:
from typing import Dict, Any, Callable
import optuna
from dataclasses import asdict

class HyperparameterSearch:
    """
    Универсальный менеджер поиска гиперпараметров для DEGANN.
    
    Инкапсулирует:
    - Пространство поиска (топология + компиляция)
    - Данные для обучения
    - Логику создания и оценки моделей
    - Треккинг лучшей модели
    
    Пример использования:
    >>> search = HyperparameterSearch(topology_config, compile_config, train_data, IModel)
    >>> best_model, best_score = run_hyperparameter_search(search)
    """
    
    def __init__(
        self,
        topology_config: Any,
        compile_config: Any,
        train_data: Dict[str, Any],
        model_factory: Callable,
        best_model_storage: Dict[str, Any] = None
    ):
        """
        Инициализация поиска.
        
        Args:
            topology_config: Базовый конфиг топологии с tunable-полями.
            compile_config: Базовый конфиг компиляции с tunable-полями.
            train_data: Словарь с данными {"X_train": ..., "y_train": ..., ...}.
            model_factory: Функция создания модели (например, IModel).
            best_model_storage: Опциональное хранилище для лучшей модели.
        """
        self.topology_sampler = ConfigSampler(topology_config)
        self.compile_sampler = ConfigSampler(compile_config)
        self.train_data = train_data
        self.model_factory = model_factory
        self.best_model_data = best_model_storage or {
            "best_model": None, 
            "best_score": float("inf"),
            "best_configs": None
        }

    def generate_configs(self, trial: optuna.Trial) -> Dict[str, Any]:
        """Генерирует связанные конфиги топологии и компиляции."""
        return {
            "topology": generate_sample(trial, self.topology_sampler),
            "compile": generate_sample(trial, self.compile_sampler)
        }

    def train_model(self, configs: Dict[str, Any]) -> float:
        """Обучает модель и возвращает метрику качества."""
        model = self.model_factory(configs["topology"])
        configs["compile"].add_eval_metric("root_mean_squared_error")
        model.compile(configs["compile"])
        
        history = model.train(
            self.train_data["X_train"],
            self.train_data["y_train"],
            epochs=self.train_data.get("epochs", 100),
            mini_batch_size=self.train_data.get("batch_size", 64),
            verbose=self.train_data.get("verbose", 0)
        )
        
        val_loss = model.evaluate(
            self.train_data["X_test"],
            self.train_data["y_test"],
            verbose=0,
            return_dict=True
        )["root_mean_squared_error"]
        
        if val_loss < self.best_model_data["best_score"]:
            self.best_model_data.update({
                "best_model": model,
                "best_score": val_loss,
                "best_configs": configs
            })
        
        return val_loss

    def get_objective(self) -> Callable[[optuna.Trial], float]:
        """Возвращает функцию для оптимизации в Optuna."""
        def objective(trial: optuna.Trial) -> float:
            configs = self.generate_configs(trial)
            return self.train_model(configs)
        return objective


**Преимущества подхода:**  
1. **Гибкость**  
   - Поддержка любых архитектур через `model_factory`  
   ```python
   model_factory=lambda cfg: IModel(cfg)  # Или кастомный конструктор
   ```  
2. **Расширяемость**  
   - Добавление новых параметров через FieldMetadata  
3. **Сквозное тестирование**  
   ```python
   # Тест проверяет весь пайплайн:
   search = HyperparameterSearch(...)
   study = optuna.create_study()
   study.optimize(search.get_objective(), n_trials=10)
   ```



---

### 🧪 **Тестирование концепции - GAN как пример**

**Проверяем:**  
1. Корректность интеграции всех компонентов  
2. Работоспособность на реальной архитектуре  



In [17]:
# 1. Подготовка данных и конфигов
train_data = {
    "X_train": X_train, "y_train": y_train,
    "X_test": X_test, "y_test": y_test,
    "epochs": 20, "batch_size": 32
}

# Для генератора
gen_config = DenseNetParams(
    input_size=1,
    block_size=[3] * 3,
    output_size=1,
    activation_func="leaky_relu",
    # Добавляем metadata для tunable полей
    metadata={
        "block_size": FieldMetadata(
            choices=[
                [16, 16],       # 2 слоя
                [32, 64, 32],    # 3 слоя
                [16, 32, 32, 16] # 4 слоя
            ]
        ),
        "activation_func": FieldMetadata(
            choices=["relu", "leaky_relu", "sigmoid", "tanh"]
        )
    }
)

# Для дискриминатора (аналогично)
disc_config = DenseNetParams(
    input_size=2,
    block_size=[3] * 3,
    output_size=1,
    activation_func="leaky_relu",
    metadata={
        "block_size": FieldMetadata(
            choices=[
                [16, 16],       # 2 слоя
                [32, 64, 32],    # 3 слоя
                [16, 32, 32, 16] # 4 слоя
            ]
        ),
        "activation_func": FieldMetadata(choices=["relu", "leaky_relu", "sigmoid"])
    }
)

base_topology_config = GANTopologyParams(
    generator_params=gen_config,
    discriminator_params=disc_config,
)


learning_rate = 1e-4
# Компиляция
gen_compile_config = DenseNetCompileParams(
    rate=learning_rate,
    optimizer="Adam",
    loss_func="BinaryCrossentropy",
    metric_funcs=["mean_absolute_error"],
)
    
disc_compile_config = DenseNetCompileParams(
    rate=learning_rate,
    optimizer="Adam",
    loss_func="BinaryCrossentropy",
    metric_funcs=["binary_accuracy"],
)
  
base_compile_config = GANCompileParams(
    generator_params=gen_compile_config,
    discriminator_params=disc_compile_config,
)



**Запуск поиска по данным:**

In [18]:
# 2. Инициализация поиска
search = HyperparameterSearch(
    topology_config=base_topology_config,
    compile_config=base_compile_config,
    train_data=train_data,
    model_factory=lambda cfg: IModel(cfg)  # Или кастомный конструктор
)



# 3. Запуск Optuna
study = optuna.create_study(direction="minimize")
study.optimize(search.get_objective(), n_trials=20)

# 4. Доступ к лучшей модели
best_m_h_s = search.best_model_data["best_model"]
print("Best score:", search.best_model_data["best_score"])

[I 2025-06-08 14:39:31,773] A new study created in memory with name: no-name-a5001b36-e7e0-4e9e-918a-a77ff9e8df1d
c:\Users\Mikhail\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend\tensorflow\nn.py:674: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
[I 2025-06-08 14:39:35,017] Trial 0 finished with value: 1.3016501665115356 and parameters: {'generator_params.block_size': [16, 16], 'generator_params.activation_func': 'tanh', 'discriminator_params.block_size': [16, 32, 32, 16], 'discriminator_params.activation_func': 'sigmoid'}. Best is trial 0 with value: 1.3016501665115356.
[I 2025-06-08 14:39:37,784] Trial 1 finished with value: 0.972818911075592 and parameters: {'generator_params.block_size': [16, 16], 'generator_params.activation_func': 'tanh', 'discriminator_params.block_size': [16, 16]

Best score: 0.6769137978553772



**Выводы по тесту:**  
✅ **Рабочий пайплайн**  
- Optuna → ConfigSampler → IModel работает без ошибок  
- Лучшая модель сохраняется автоматически  



---

**Зададим функцию обобщенную:**   
🧩 **Она будет принимать:**
1) HyperparameterSearch
2) политику поиска 

🧩 **Возвращать:**  
1) нейросеть
2) ошибку на валидационных данных

Этой функцией примерно покажем как выглядит готовый результат системы Optuna + ConfigSampler

In [19]:
from typing import Dict, Any, Callable, Tuple
import optuna
from optuna.samplers import BaseSampler

def run_hyperparameter_search(
    search: HyperparameterSearch,
    search_policy: Any = None,  # Может быть Optuna-семплер или кастомный объект
    n_trials: int = 100,
    direction: str = "minimize"
) -> Tuple[Any, float]:
    """
    Запускает поиск гиперпараметров и возвращает лучшую модель и её ошибку.

    Аргументы:
        search: Экземпляр HyperparameterSearch.
        search_policy: Политика поиска (например, optuna.samplers.TPESampler()).
        n_trials: Количество trials.
        direction: Направление оптимизации ("minimize" или "maximize").

    Возвращает:
        Лучшая модель и её ошибка (например, val_loss).
    """
    # Инициализация Optuna
    study = optuna.create_study(
        direction=direction,
        sampler=search_policy  # Например, TPESampler() или RandomSampler()
    )
    
    # Запуск оптимизации
    study.optimize(search.get_objective(), n_trials=n_trials)

    # Возвращаем лучшую модель и её ошибку
    return (
        search.best_model_data["best_model"],
        search.best_model_data["best_score"]
    )

**Небольшой тест, который показывает как выглядит вызов:**

In [20]:
from optuna.samplers import TPESampler

# Пример: Запуск поиска для GAN  
best_m_r_h_s, best_score = run_hyperparameter_search(  
    search,
    search_policy=TPESampler(),  
    n_trials=20 
)  
print(f"Лучшая ошибка: {best_score:.4f}")  


[I 2025-06-08 14:40:40,205] A new study created in memory with name: no-name-9cfc5291-dde4-48ae-b2a5-d920cfc9bc46
[I 2025-06-08 14:40:43,291] Trial 0 finished with value: 0.6945288181304932 and parameters: {'generator_params.block_size': [32, 64, 32], 'generator_params.activation_func': 'sigmoid', 'discriminator_params.block_size': [16, 16], 'discriminator_params.activation_func': 'sigmoid'}. Best is trial 0 with value: 0.6945288181304932.
[I 2025-06-08 14:40:46,534] Trial 1 finished with value: 0.7578610777854919 and parameters: {'generator_params.block_size': [16, 32, 32, 16], 'generator_params.activation_func': 'sigmoid', 'discriminator_params.block_size': [16, 16], 'discriminator_params.activation_func': 'relu'}. Best is trial 0 with value: 0.6945288181304932.
[I 2025-06-08 14:40:49,603] Trial 2 finished with value: 0.8316156268119812 and parameters: {'generator_params.block_size': [16, 16], 'generator_params.activation_func': 'tanh', 'discriminator_params.block_size': [32, 64, 32]

Лучшая ошибка: 0.6741


In [21]:

# Вывод параметров лучшей модели  
if best_m_r_h_s:  
    print(best_m_r_h_s)
  

IModel tensorflow_dense_net_106
Layer TFDense0
weights shape = (1, 16)
Layer TFDense1
weights shape = (16, 16)
Layer OutLayerTFDense
weights shape = (16, 1)


IModel tensorflow_dense_net_107
Layer TFDense0
weights shape = (2, 16)
Layer TFDense1
weights shape = (16, 16)
Layer OutLayerTFDense
weights shape = (16, 1)



**Так же покажем что работает не только с gan но и с полносвязной сетью!**

In [22]:
densnet_topology_config = DenseNetParams(
    input_size=1,
    block_size=[3] * 3,
    output_size=1,
    activation_func="leaky_relu",
    metadata={ # указание того что можно перебирать реализуется через metadata.
        "block_size": FieldMetadata(
            choices=[
                [2, 4, 8, 4, 2],
                [16, 16],       # 2 слоя
                [32, 64, 32],    # 3 слоя
                [16, 32, 32, 16] # 4 слоя
            ]
        ),
        "activation_func": FieldMetadata(choices=["relu", "leaky_relu", "sigmoid"])
    }
)

densnet_compile_config = DenseNetCompileParams(
    rate=1e-4,
    optimizer="Adam",
    loss_func="BinaryCrossentropy",
    metric_funcs=["binary_accuracy"],
)

In [23]:
from optuna.samplers import CmaEsSampler

densnet_search = HyperparameterSearch(
    topology_config=densnet_topology_config,
    compile_config=densnet_compile_config,
    train_data=train_data,
    model_factory=lambda cfg: IModel(cfg)  # Или кастомный конструктор
)


# Пример: Запуск поиска для GAN  
best_m_r_h_s_densenet, best_score = run_hyperparameter_search(  
    densnet_search,
    search_policy=CmaEsSampler(),  # Можем использовать отличную от прошлого теста политику поиска
    n_trials=20
)  
print(f"Лучшая ошибка: {best_score:.4f}")  
print(best_m_r_h_s_densenet.evaluate(X_test, y_test, verbose=0, return_dict=True))

[I 2025-06-08 14:41:45,871] A new study created in memory with name: no-name-b246e270-a54d-4abd-a57b-f3fa850b6732
c:\Users\Mikhail\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [2, 4, 8, 4, 2] which is of type list.
  warnings.warn(message)
[I 2025-06-08 14:41:47,872] Trial 0 finished with value: 0.6760132312774658 and parameters: {'block_size': [32, 64, 32], 'activation_func': 'sigmoid'}. Best is trial 0 with value: 0.6760132312774658.
[W 2025-06-08 14:41:47,873] The parameter 'block_size' in trial#1 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsS

Лучшая ошибка: 0.6191
{'binary_accuracy': 0.0, 'loss': 0.6777036786079407, 'root_mean_squared_error': 0.6191056966781616}


In [24]:

# Вывод параметров лучшей модели  
if best_m_r_h_s_densenet:  
    print(best_m_r_h_s_densenet)


IModel tensorflow_dense_net_144
Layer TFDense0
weights shape = (1, 16)
Layer TFDense1
weights shape = (16, 32)
Layer TFDense2
weights shape = (32, 32)
Layer TFDense3
weights shape = (32, 16)
Layer OutLayerTFDense
weights shape = (16, 1)




---

## 🛠️ Реализация пользовательского алгоритма поиска (Simulated Annealing)

### Контекст и проблема
**Исходная сложность:**
- Алгоритмы поиска были жестко привязаны к конкретным топологиям DEGANN.
- Добавление новых алгоритмов требовало модификации кода под каждую архитектуру.

**Решение:**
- Использование `ConfigSampler` как прослойки между алгоритмами поиска и конфигами DEGANN.
- Алгоритмы работают только с "плоскими" параметрами (словарь имён и диапазонов), не зная о внутренней структуре моделей.

---

### Пример: Simulated Annealing Sampler для Optuna
**Ключевые особенности реализации:**
- Наследуется от `BaseSampler` Optuna.
- Полностью абстрагирован от конкретных топологий.
- Работает через стандартный интерфейс `sample_relative`/`sample_independent`.


In [25]:
import optuna
import numpy as np
import math
from optuna.samplers import BaseSampler
from optuna.distributions import BaseDistribution, FloatDistribution, IntDistribution, CategoricalDistribution
from typing import Dict, Any, Optional, Sequence, List
from dataclasses import is_dataclass, fields


class SimulatedAnnealingSampler(BaseSampler):
    def __init__(self, 
                 initial_temperature: float = 10.0,
                 cooling_factor: float = 0.95,
                 reannealing_interval: int = 100,
                 seed: Optional[int] = None):
        self.initial_temperature = initial_temperature
        self.current_temperature = initial_temperature
        self.cooling_factor = cooling_factor
        self.reannealing_interval = reannealing_interval
        self.rng = np.random.RandomState(seed)
        self._best_value = None
        self._best_params = None
        self._trial_count = 0

    def infer_relative_search_space(self, study: "optuna.study.Study", 
                                  trial: "optuna.trial.FrozenTrial") -> Dict[str, BaseDistribution]:
        return {}

    def sample_relative(self, study: "optuna.study.Study", 
                       trial: "optuna.trial.FrozenTrial", 
                       search_space: Dict[str, BaseDistribution]) -> Dict[str, Any]:
        if len(study.trials) == 0:
            return {}

        # Реаннилинг (сброс температуры)
        if self._trial_count % self.reannealing_interval == 0:
            self.current_temperature = self.initial_temperature
        
        self._trial_count += 1
        
        last_trial = study.trials[-1]
        last_value = last_trial.value
        last_params = last_trial.params
        
        # Принятие лучшего решения
        if self._best_value is None or \
           (study.direction == optuna.study.StudyDirection.MAXIMIZE and last_value > self._best_value) or \
           (study.direction == optuna.study.StudyDirection.MINIMIZE and last_value < self._best_value):
            self._best_value = last_value
            self._best_params = last_params
            return {}
        
        # Вероятностное принятие худшего решения
        delta = last_value - self._best_value
        if study.direction == optuna.study.StudyDirection.MAXIMIZE:
            delta = -delta
            
        acceptance_prob = math.exp(-delta / self.current_temperature)
        if self.rng.random() < acceptance_prob:
            self._best_value = last_value
            self._best_params = last_params
            return {}
        else:
            return self._best_params

    def sample_independent(self, study: "optuna.study.Study", 
                          trial: "optuna.trial.FrozenTrial", 
                          param_name: str, 
                          param_distribution: BaseDistribution) -> Any:
        if isinstance(param_distribution, FloatDistribution):
            if param_distribution.log:
                log_low = math.log(param_distribution.low)
                log_high = math.log(param_distribution.high)
                return math.exp(self.rng.uniform(log_low, log_high))
            else:
                return self.rng.uniform(param_distribution.low, param_distribution.high)
        elif isinstance(param_distribution, IntDistribution):
            return self.rng.randint(param_distribution.low, param_distribution.high + 1)
        elif isinstance(param_distribution, CategoricalDistribution):
            choices = param_distribution.choices
            # Обработка случая, когда choices содержит сложные объекты
            if len(choices) > 0 and (is_dataclass(choices[0]) or isinstance(choices[0], (list, dict))):
                return choices[self.rng.randint(0, len(choices))]
            else:
                return self.rng.choice(choices)
        else:
            raise ValueError(f"Unsupported distribution type: {type(param_distribution)}")

    def after_trial(self, study: "optuna.study.Study", 
                   trial: "optuna.trial.FrozenTrial", 
                   state: optuna.trial.TrialState, 
                   values: Optional[Sequence[float]]) -> None:
        self.current_temperature *= self.cooling_factor

**Как это устраняет проблему:**
✅ **Отделение логики поиска от топологий**  
   - Алгоритм не знает о существовании GAN/DenseNet/etc.  
   - Взаимодействует только с базовыми типами параметров (числа, категории).  

✅ **Гибкость**  
   - Один алгоритм можно применять к любым моделям DEGANN.  
   - Добавление новой архитектуры не требует изменений в коде поиска.  

---

### Тестирование интеграции
**Проверка на математической функции:**

In [26]:
def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    y = trial.suggest_float('y', -10, 10)
    return (x - 2)**2 + (y + 3)**2  # Минимизируем расстояние до точки (2, -3)

# Настройки имитации отжига
sampler = SimulatedAnnealingSampler(
    initial_temperature=100.0,  # Начальная температура
    cooling_factor=0.99,        # Коэффициент охлаждения (ближе к 1 - медленнее охлаждение)
    reannealing_interval=50,    # Частота реаннинга
    seed=42                     # Фиксированный seed для воспроизводимости
)

study = optuna.create_study(
    direction='minimize',
    sampler=sampler
)

study.optimize(objective, n_trials=500)

print(f"Лучшее значение: {study.best_value}")
print(f"Лучшие параметры: {study.best_params}")

[I 2025-06-08 14:42:29,647] A new study created in memory with name: no-name-b57925ab-d463-4b7a-9c32-2c7dfc724608
[I 2025-06-08 14:42:29,649] Trial 0 finished with value: 164.6759343739632 and parameters: {'x': -2.50919762305275, 'y': 9.014286128198322}. Best is trial 0 with value: 164.6759343739632.
[I 2025-06-08 14:42:29,650] Trial 1 finished with value: 31.701376975232193 and parameters: {'x': 4.639878836228101, 'y': 1.973169683940732}. Best is trial 1 with value: 31.701376975232193.
[I 2025-06-08 14:42:29,652] Trial 2 finished with value: 93.90302950966503 and parameters: {'x': -6.87962719115127, 'y': -6.880109593275947}. Best is trial 1 with value: 31.701376975232193.
[I 2025-06-08 14:42:29,653] Trial 3 finished with value: 224.04447394709342 and parameters: {'x': -8.83832775663601, 'y': 7.323522915498703}. Best is trial 1 with value: 31.701376975232193.
[I 2025-06-08 14:42:29,655] Trial 4 finished with value: 51.286885688277 and parameters: {'x': 2.0223002348641756, 'y': 4.161451

Лучшее значение: 0.1581052271131801
Лучшие параметры: {'x': 2.3625648091579166, 'y': -2.8367456393431905}


**Применение к поиску топологии DENSNET:**

In [27]:

# Настройки имитации отжига
sim_anneal_sampler = SimulatedAnnealingSampler(
    initial_temperature=100.0,  # Начальная температура
    cooling_factor=0.99,        # Коэффициент охлаждения (ближе к 1 - медленнее охлаждение)
    reannealing_interval=50,    # Частота реаннинга
    seed=42                     # Фиксированный seed для воспроизводимости
)

densnet_search_sim_anneal = HyperparameterSearch(
    topology_config=densnet_topology_config,
    compile_config=densnet_compile_config,
    train_data=train_data,
    model_factory=lambda cfg: IModel(cfg)  # Или кастомный конструктор
)


# Пример: Запуск поиска для GAN  
best_m_r_h_s_densenet_sim_ann, best_score = run_hyperparameter_search(  
    densnet_search_sim_anneal,
    search_policy=sim_anneal_sampler,  
    n_trials=20 
)  


print(f"Лучшая ошибка: {best_score:.4f}")  
print(f"Лучшая модель: {best_m_r_h_s_densenet_sim_ann}")

[I 2025-06-08 14:42:39,330] A new study created in memory with name: no-name-c5497257-c32f-4d94-9ef7-b13d5b44e8aa
[I 2025-06-08 14:42:41,538] Trial 0 finished with value: 0.5805098414421082 and parameters: {'block_size': [32, 64, 32], 'activation_func': 'relu'}. Best is trial 0 with value: 0.5805098414421082.
[I 2025-06-08 14:42:43,613] Trial 1 finished with value: 0.6748066544532776 and parameters: {'block_size': [32, 64, 32], 'activation_func': 'sigmoid'}. Best is trial 0 with value: 0.5805098414421082.
[I 2025-06-08 14:42:45,916] Trial 2 finished with value: 0.48260411620140076 and parameters: {'block_size': [16, 32, 32, 16], 'activation_func': 'relu'}. Best is trial 2 with value: 0.48260411620140076.
[I 2025-06-08 14:42:48,281] Trial 3 finished with value: 0.6782308220863342 and parameters: {'block_size': [2, 4, 8, 4, 2], 'activation_func': 'sigmoid'}. Best is trial 2 with value: 0.48260411620140076.
[I 2025-06-08 14:42:50,118] Trial 4 finished with value: 0.6772960424423218 and pa

Лучшая ошибка: 0.4296
Лучшая модель: IModel tensorflow_dense_net_152
Layer TFDense0
weights shape = (1, 32)
Layer TFDense1
weights shape = (32, 64)
Layer TFDense2
weights shape = (64, 32)
Layer OutLayerTFDense
weights shape = (32, 1)




---

## 🎯 Итоги и направления развития  

### **Что реализовано:**  
✅ **Универсальный поиск гиперпараметров**  
   - `ConfigSampler` автоматически определяет `tunable`-поля и преобразует их в конфиги.  
   - `HyperparameterSearch` объединяет логику обучения, оценки и трекинга лучшей модели.  
   - Интеграция с `Optuna` через `generate_sample`.  

✅ **Гибкость архитектуры**  
   - Поддержка любых моделей DEGANN (GAN, DenseNet и др.) через `model_factory`.  
   - Работает не только с GAN, но и с другими архитектурами (например, DenseNet), что подтверждено тестами.  

✅ **Интеграция пользовательских алгоритмов поиска**  
   - Пример с `SimulatedAnnealingSampler` показал возможность добавления новых алгоритмов без изменения кода топологий.  
   - Алгоритмы работают через стандартный интерфейс, абстрагируясь от конкретных реализаций моделей.  

### **Результаты экспериментов:**  
🔹 **Для GAN:**  
   - Успешная оптимизация архитектуры (слои, нейроны, LR).  
   - Автоматическое сохранение лучшей модели.  

🔹 **Для DenseNet:**  
   - Корректный подбор параметров (например, `block_size` и функции активации).  
   - Подтверждена работоспособность на реальных данных.  

🔹 **Для Simulated Annealing:**  
   - Успешная интеграция алгоритма в систему.  
   - Продемонстрирована гибкость системы при добавлении новых методов поиска.  

### **Что можно улучшить:**  
🔧 **Динамические диапазоны**  
   - Реализация `SmartRange` для генерации сложных структур (например, `block_size` с переменной длиной).  
   - Автоматическая декомпозиция в "плоские" параметры для алгоритмов поиска.  

🔧 **Расширение поддержки алгоритмов**  
   - Добавление других оптимизаторов (например, `CMA-ES` без жёсткой привязки к `Optuna`).  

🔧 **Упрощение описания топологий**  
   - Более удобный синтаксис для `tunable`-параметров (например, через декораторы).  

🔧 **Повышение модульности**  
   - Чёткое разделение модулей и устранение избыточных зависимостей.  
   - Реализация интерфейса для добавления новых систем поиска (по аналогии с Optuna).  

---  

**Вывод:**  
Архитектурное решение доказало свою жизнеспособность. Система позволяет:  
- Гибко настраивать поиск гиперпараметров для разных топологий.  
- Интегрировать новые алгоритмы без изменения кода моделей.  
- Масштабироваться для поддержки сложных сценариев оптимизации.  

**Дальнейшая работа:**  
1. Повышение модульности:  
   - Устранение оставшихся зависимостей от ядра DEGANN.  
   - Полное абстрагирование через слой `ConfigSampler`.  
2. Реализация интеграции (вне формата `.ipynb`).  
3. Внедрение динамических диапазонов (при одобрении).  
4. Крупномасштабное тестирование, демонстрирующее валидность решения с точки зрения эффективности поиска, а не только корректной работоспособности.
5. Расширение библиотеки алгоритмов.  
